In [1]:
from imports import *

In [2]:
overview = pd.read_csv('game_overview.csv')
data = pd.read_csv('train.csv')
overview.head()

,title,developer,publisher,tags,overview
0,Spooky's Jump Scare Mansion,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
1,Sakura Clicker,Winged Cloud,Winged Cloud,"['Nudity', 'Anime', 'Free to Play', 'Mature', ...",The latest entry in the Sakura series is more ...
2,WARMODE,WARTEAM,WARTEAM,"['Early Access', 'Free to Play', 'FPS', 'Multi...",Free to play shooter about the confrontation o...
3,Fractured Space,Edge Case Games Ltd.,Edge Case Games Ltd.,"['Space', 'Multiplayer', 'Free to Play', 'PvP'...",Take the helm of a gigantic capital ship and g...
4,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,"['FPS', 'Multiplayer', 'Shooter', 'Action', 'T...",Counter-Strike: Global Offensive (CS: GO) expa...


In [3]:
data.head()

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1


In [4]:
df = data.merge(overview[['title', 'tags']], on = 'title')[['title', 'tags', 'user_review']]
df.head()

,title,tags,user_review
0,Spooky's Jump Scare Mansion,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",I'm scared and hearing creepy voices. So I'll...
1,Spooky's Jump Scare Mansion,"['Horror', 'Free to Play', 'Cute', 'First-Pers...","Best game, more better than Sam Pepper's YouTu..."
2,Spooky's Jump Scare Mansion,"['Horror', 'Free to Play', 'Cute', 'First-Pers...","A littly iffy on the controls, but once you kn..."
3,Spooky's Jump Scare Mansion,"['Horror', 'Free to Play', 'Cute', 'First-Pers...","Great game, fun and colorful and all that.A si..."
4,Spooky's Jump Scare Mansion,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Not many games have the cute tag right next to...


In [5]:
def clean(review):
    review = review.lower()
    review = re.sub(r'[^a-zA-Z0-9 ]', '', review)
    return review

def concate(row):
    valid = ['horror', 'rpg', 'shooter', 'strategy', 'sports', 'rougelite', 'rougelike', 'open world', 'fighting']
    for genre in row:
        genre = genre.lower()
        genre = genre.strip('[]" \'')
        if genre in valid:
            if genre == 'rougelike':
                return 'rougelite'
            return genre
    return 'Misc'
# list(overview['tags'].apply(clean).str.split().explode().unique())
df = data.merge(overview[['title', 'tags']], on = 'title')[['title', 'tags', 'user_review']]
df['tags'] = df['tags'].apply(ast.literal_eval).apply(concate)
df['user_review'] = df['user_review'].apply(clean)
df.head()

,title,tags,user_review
0,Spooky's Jump Scare Mansion,horror,im scared and hearing creepy voices so ill pa...
1,Spooky's Jump Scare Mansion,horror,best game more better than sam peppers youtube...
2,Spooky's Jump Scare Mansion,horror,a littly iffy on the controls but once you kno...
3,Spooky's Jump Scare Mansion,horror,great game fun and colorful and all thata side...
4,Spooky's Jump Scare Mansion,horror,not many games have the cute tag right next to...


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['user_review'], df['tags'], test_size=0.2)

# pipeline = make_pipeline(TfidfVectorizer(stop_words='english', max_features=5000),
#     MultinomialNB()
# )

# param_grid = {
#     'tfidfvectorizer__max_features': [1000, 3000, 5000, None],
#     'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
#     'tfidfvectorizer__use_idf': [True, False],
#     'tfidfvectorizer__smooth_idf': [True, False],
#     'tfidfvectorizer__sublinear_tf': [True, False],
#     'tfidfvectorizer__stop_words': ['english', None],

#     # MultinomialNB parameters
#     'multinomialnb__alpha': [0.01, 0.1, 0.5, 1.0, 5.0],
#     'multinomialnb__fit_prior': [True, False]
# }

# # Set up GridSearchCV
# grid_search = GridSearchCV(pipeline, param_grid=param_grid, 
#                            cv=5, scoring='accuracy', n_jobs=-1)

# grid_search.fit(X_train, y_train)


In [7]:
improve_pipeline = make_pipeline(
    TfidfVectorizer(stop_words='english', max_features=5000),
    MultinomialNB(alpha= 0.5, fit_prior= False)
)
improve_pipeline.fit(X_train, y_train)

y_test_pred = improve_pipeline.predict(X_test)

print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Test Classification Report:
              precision    recall  f1-score   support

    fighting       0.53      0.68      0.59        75
      horror       0.74      0.89      0.81        82
  open world       0.78      0.81      0.79       691
         rpg       0.74      0.80      0.77       495
     shooter       0.86      0.87      0.86      1104
      sports       0.72      0.77      0.75        97
    strategy       0.92      0.79      0.85       955

    accuracy                           0.82      3499
   macro avg       0.75      0.80      0.77      3499
weighted avg       0.83      0.82      0.82      3499



In [8]:
joblib.dump(improve_pipeline, 'genre_classifier_model.joblib')
loaded_model = joblib.load('genre_classifier_model.joblib')
loaded_model

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=5000, stop_words='english')),
                ('multinomialnb', MultinomialNB(alpha=0.5, fit_prior=False))])

In [9]:
df.to_csv('final_data.csv')